In [1]:
from skdata.mnist.views import OfficialVectorClassification
from tqdm import tqdm
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
data = OfficialVectorClassification()

In [3]:
# Labels
df_labels = pd.get_dummies(data.all_labels)
df_labels.shape

(70000, 10)

In [4]:
# Images
features = data.all_vectors
features.shape

(70000, 784)

In [5]:
LIMIT = 1000

In [6]:
features = np.array(features)
labels = np.array(df_labels)

features = features[:LIMIT]
labels = labels[:LIMIT]


# Reshape the data from rows to columns
features = features.reshape(features.shape[0], 784, 1)
labels = labels.reshape(labels.shape[0], 10)

In [7]:
# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
num_steps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

# Parameters
learning_rate = 0.001

In [8]:
data = tf.placeholder(tf.float32, [None, 784, 1])
target = tf.placeholder(tf.float32, [None, 10])

In [9]:
num_hidden = 30

cell = tf.nn.rnn_cell.LSTMCell(num_hidden, state_is_tuple=True)
val, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)

# TODO: Understand what happens in the next couple of lines!!!
val = tf.transpose(val, [1, 0, 2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)

weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.0, shape=[target.get_shape()[1]]))

In [10]:
# Functions definition 

prediction = tf.matmul(last, weight) + bias

loss = tf.sqrt(tf.reduce_mean(tf.square(target - prediction)), name='RMSE')
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
minimize = optimizer.minimize(loss)

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(target, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

/usr/local/lib/python2.7/site-packages/tensorflow/python/ops/gradients.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [11]:
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

In [12]:
batch_size = 100
no_of_batches = int(len(features) / batch_size)
epochs = 400

loss_train_array = []
train_accuracy_array = []
test_accuracy_array = []

10


In [25]:
with sess.as_default():
    
    for i in range(epochs):
        
        ptr = 0
        for j in range(no_of_batches):
            inp, out = features[ptr:ptr + batch_size], labels[ptr:ptr + batch_size]
            ptr += batch_size
            _, loss_train = sess.run([minimize, loss], {data: inp, target: out})
        
        if i % 10 == 0:
            # We calculate the accuracies to plot their values later
            loss_train_array.append(loss_train)
            train_accuracy = sess.run(
                accuracy, feed_dict={data: features[:10], target: labels[:10]})
            train_accuracy_array.append(train_accuracy)

            print (
                'Epoch %04d, '
                'loss train %.8f, train accuracy %.8f'
                %
                (i,
                 loss_train,
                 train_accuracy))

Epoch 0000, loss train 0.29055846, train accuracy 0.30000001
Epoch 0010, loss train 0.28459078, train accuracy 0.40000001
Epoch 0020, loss train 0.28086957, train accuracy 0.40000001
Epoch 0030, loss train 0.27600390, train accuracy 0.60000002
Epoch 0040, loss train 0.31244469, train accuracy 0.40000001
Epoch 0050, loss train 0.27201024, train accuracy 0.50000000
Epoch 0060, loss train 0.28449625, train accuracy 0.10000000
Epoch 0070, loss train 0.28073186, train accuracy 0.10000000
Epoch 0080, loss train 0.28070363, train accuracy 0.10000000
Epoch 0090, loss train 0.27909574, train accuracy 0.00000000
Epoch 0100, loss train 0.28013858, train accuracy 0.20000000
Epoch 0110, loss train 0.27588651, train accuracy 0.10000000
Epoch 0120, loss train 0.27426177, train accuracy 0.50000000
Epoch 0130, loss train 0.28004283, train accuracy 0.20000000
Epoch 0140, loss train 0.26907411, train accuracy 0.20000000
Epoch 0150, loss train 0.27606848, train accuracy 0.60000002
Epoch 0160, loss train 0

In [34]:
image_test = features[:1]
label_test = labels[:1]
label_test

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]])

In [35]:
# Train predicted
pred = sess.run(prediction ,{data: image_test})
print pred

[[ 0.125304    0.09261802  0.15892717  0.10722232  0.05070642  0.16740942
   0.1376359  -0.01782649  0.13625304 -0.01044179]]
